In [68]:
with open('test_input.txt') as file:
    data = file.read()

data = data.split('\n')
data = ['A' + row for row in data]
mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'A':10}
numbers = [int(row[1:-1]) for row in data]
inputs = [[mapping[x] for x in row] for row in data]

In [69]:


keypad = {7: (0, 0), 8: (0, 1),9: (0, 2),4: (1, 0),5: (1, 1),6: (1, 2),1: (2, 0),2: (2, 1),3: (2, 2),0: (3, 1),10: (3, 2)}
directional_keypad = {0: (0,1), 1: (1,0), 2:(1,1), 3:(1,2), 4:(0,2)}
def key_presses(prev_kp,next_kp):
    # returns up, right left, down, right
    return ([4] + max(prev_kp[0]-next_kp[0],0) * [0] + max(next_kp[1]-prev_kp[1],0) * [3]  + max(next_kp[0]-prev_kp[0],0) * [2] + max(prev_kp[1]-next_kp[1],0) * [1]  + [4]) 

example1 = 10,0,2,9,10
example1 = 10,9,8,0,10


complex_score = 0
for input_kp,number in zip(inputs,numbers):
    s = 0
    for ii in range(len(input_kp)-1):
        kps1 = key_presses(keypad[input_kp[ii]],keypad[input_kp[ii+1]])
        # print(kps1,  'First keypad')
        for jj in range(len(kps1)-1):
            kps2 = key_presses(directional_keypad[kps1[jj]],directional_keypad[kps1[jj+1]])
            # print(' '*5, kps2, 'Second keypad')
            for kk in range(len(kps2)-1):
                kps3 = key_presses(directional_keypad[kps2[kk]],directional_keypad[kps2[kk+1]])
                s += len(kps3)-1
                # print(' '*10,kps3)
    complex_score += s*number
    print(number,s)


complex_score




29 68
980 60
179 68
456 64
379 68


127900